# COMP 551 AML Assignment 2
## Ran Cheng, 260768706
This project is for assignment 2 of applied machine learning course.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# read classifier data, mean
ds1_m0_data = pd.read_csv('./Data/DS1_m_0.txt', sep=",", header=None)
ds1_m0_data = ds1_m0_data.dropna(axis=1, how='all')
ds1_m0 = np.squeeze(np.asarray(ds1_m0_data))
ds1_m1_data = pd.read_csv('./Data/DS1_m_1.txt', sep=",", header=None)
ds1_m1_data = ds1_m1_data.dropna(axis=1, how='all')
ds1_m1 = np.squeeze(np.asarray(ds1_m1_data))
print(ds1_m0, ds1_m1)

[1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3
 1.3 1.3] [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [3]:
# read covarience matrrix
ds1_cov_data = pd.read_csv('./Data/DS1_Cov.txt', sep=",", header=None)
ds1_cov_data = ds1_cov_data.dropna(axis=1, how='all')
ds1_cov = np.squeeze(np.asarray(ds1_cov_data))
ds1_cov.shape

(20, 20)

In [4]:
# get the data for classifier, 0
out_0 = np.random.multivariate_normal(ds1_m0, ds1_cov, size=2000)
label_0 = np.zeros((2000,1))
labeled_ds1_0 = np.hstack((out_0, label_0))
# get the data for classifier, 1
out_1 = np.random.multivariate_normal(ds1_m1, ds1_cov, size=2000)
label_1 = np.ones((2000,1))
labeled_ds1_1 = np.hstack((out_1, label_1))
print(labeled_ds1_0[0], labeled_ds1_1[1])

[ 1.22670522  1.11875031  2.10584674  0.42686339 -0.38069006  2.31383079
  2.94942502  2.60081352  1.75815284  5.35586347  1.36733279  1.03294138
  0.89712872  0.5713176   1.40034432  3.88254911 -0.36160315  1.90975477
  0.11680028  0.74042753  0.        ] [-0.16501258 -0.27942621 -1.16703488 -0.99159389 -1.20104302 -0.8192157
  0.86858126  0.69181763  1.28661494 -1.10579824  0.46289595  0.82346982
 -1.47932827 -0.78399104 -1.06281244  0.1551596   0.63767731 -0.44418858
  0.99120889 -2.55542526  1.        ]


In [5]:
# concaternate negative and positive data
merged_ds1 = np.vstack((labeled_ds1_0,labeled_ds1_1))
# shuffle the negative and positive data
np.random.shuffle(merged_ds1)
# get the data split into two parts randomly: training and testing data
train, test = train_test_split(merged_ds1, test_size=0.3)
n = train.shape[0]
n_test = test.shape[0]

In [6]:
mu1 = np.mean(train[train[0:,-1] == 1][0:,0:-1], axis=0) #mu1 hat
mu0 = np.mean(train[train[0:,-1] == 0][0:,0:-1], axis=0) #mu1 hat
mu0.shape

(20,)

In [7]:
n_0 = len(train[train[0:,-1] == 0])
n_1 = len(train[train[0:,-1] == 1])
p_y0 = n_0/n
p_y1 = n_1/n
print('n_0: % d,' % n_0, ' n_1: % d' % n_1)

n_0:  1403,  n_1:  1397


In [8]:
s1 = train[train[0:,-1] == 1][0:, 0:-1] - mu1
sig1 =  np.dot(s1.transpose(), s1)
s0 = train[train[0:,-1] == 1][0:, 0:-1] - mu0
sig0 =  np.dot(s0.transpose(), s0)
sigma = (sig0 + sig1)/n
invsig = np.linalg.inv(sigma)

In [21]:
# estimate w0 and w1
w_0 = np.log(p_y0) - np.log(p_y1) - (1/2)*(np.dot(np.dot(mu0.transpose(), invsig), mu0)) + (1/2)*(np.dot(np.dot(mu1.transpose(), invsig), mu1))
w_1 = np.dot(invsig, (mu0 - mu1))
print(w_0, w_1.shape)

4.012836542242569 (20,)


In [28]:
tmp = np.dot(test[0:,0:-1], w_1)
result = w_0 + tmp
result[result >= 0] = 0
result[result < 0] = 1
error = sum(result != test[:,-1])
print("The error rate of the estimation on the testing set is {0}/{1} or : {2}%"\
          .format(error, n_test, 100*error/n_test))

The error rate of the estimation on the testing set is 58/1200 or : 4.833333333333333%
